# Universidad del Desarrollo
## Magister Data Science - Macroeconomía
### Grupo N°2: (Región de los Lagos)
* Jose Pedro
* Cristian
* Paul
* Max
* Hernan

## Calculo Indice Precios al Consumidor

**Objetivo**

El objetivo de este notebook es el calculo de IPC según el siguiente algortimo.



$ 
\begin{equation*}
IPC_t   = |\sum_{k=1}^N \frac{p_i^t}{p_i^0} w_i| * 100
\end{equation*} 
$

Donde:
* $ p_i^t $ = Precio del producto i en el momento t
* $ p_i^0 $ = Precio del producto 0 en el mes base **(2014/08=100)**
* $ w_i   $ = Ponderación del producto i en la canasta
* N = Numero de productos

In [19]:
import pandas as pd
import numpy as np
import calendar
import datetime

%matplotlib inline

In [20]:
base = "2014-08"

### 1. Se cargan el dataset de productos

In [21]:
df = pd.read_csv('output/precios_productos.csv')

In [22]:
df.sample(10)

,Momento,Sector,Producto,Factor
1573,2017-04-01,Puerto Montt,Carnes,1.025511
1272,2016-03-01,Osorno,Hortaliza,0.960416
2317,2017-12-01,Osorno,Harina,0.965960
1102,2017-04-01,Osorno,Frutas,0.906476
2362,2014-10-01,Osorno,Aceite,0.988659
1723,2015-11-01,Puerto Montt,Azucar,1.005647
1097,2017-03-01,Osorno,Frutas,1.036203
444,2018-01-01,Puerto Varas,Huevos,1.002547
452,2018-03-01,Osorno,Huevos,0.973153
1163,2018-04-01,Puerto Montt,Frutas,0.956392


In [23]:
df.Sector.value_counts()

Frutillar       517
Osorno          517
Puerto Varas    517
Puerto Montt    517
Llanquihue      517
Name: Sector, dtype: int64

In [24]:
df_w = pd.read_csv("output/w_ponderados.csv")

In [25]:
df_w.head(11)

,Producto,Ponderacion,Ponderacion_Nueva
0,Aceite,0.34550,0.025874
1,Azucar,0.18716,0.014016
2,Carnes,4.38211,0.328176
3,Cereales,0.67947,0.050885
4,Frutas,1.18325,0.088614
5,Harina,0.09708,0.007270
6,Hortaliza,1.69038,0.126593
7,Huevos,0.33391,0.025007
8,Lacteos,2.22952,0.166969
9,Legumbre,0.13135,0.009837


### 2. Calculamos precios base

In [26]:
dfp = pd.read_csv("output/canasta_ipc.csv")

In [27]:
dfp.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08,Frutillar,Aceite,Maravilla,1807.484158
1,2014-09,Frutillar,Aceite,Maravilla,1680.222200
2,2014-10,Frutillar,Aceite,Maravilla,1651.904700
3,2014-11,Frutillar,Aceite,Maravilla,1807.484158
4,2014-12,Frutillar,Aceite,Maravilla,1807.484158


In [28]:
df_pb = dfp[dfp.Momento==base]

In [29]:
df_pb.sample(5)

,Momento,Sector,Producto,Variedad,Promedio
3369,2014-08,Puerto Montt,Carnes,Choclillo,5229.060148
13151,2014-08,Osorno,Carnes,Tapapecho,4539.630900
19278,2014-08,Puerto Montt,Frutas,Naranja,725.000000
22523,2014-08,Frutillar,Harina,Harina sin polvos de hornear,763.721525
26186,2014-08,Puerto Montt,Hortaliza,Haba,1159.243587


In [30]:
pb = df_pb.groupby(["Producto"]).mean()
pb

,Promedio
Producto,
Aceite,1550.660213
Azucar,626.972709
Carnes,4988.965919
Cereales,836.436504
Frutas,1436.414083
Harina,719.486763
Hortaliza,916.972072
Huevos,1844.462405
Lacteos,3281.595206


### 3. IPC Osorno

In [31]:
df_osorno = df[(df.Sector=="Osorno") & (df.Momento > base + "-01")]

In [32]:
df_osorno.Sector.value_counts()

Osorno    506
Name: Sector, dtype: int64

In [37]:
def GetIPC(df, df_base, df_p):
    v = []
    for o in df.Momento:
        ipc = 1
        prod = df[df.Momento == o].Producto
        for row in prod:
            p_0 = df_base.loc[row].iloc[0]
            p_j = df[(df.Momento == o) & (df.Producto == row)].Factor.iloc[0]

            w_i = df_p[df_p.Producto == row]["Ponderacion_Nueva"].iloc[0]
            
            ipc =+ ((p_j/p_0)*w_i)

        v.append({"ipc": ipc*100, "Momento": o})
        
    return pd.DataFrame(v)

In [38]:
df_ipc = GetIPC(df_osorno, pb, df_w)

In [39]:
df_ipc.head()

,Momento,ipc
0,2014-09-01,0.001639
1,2014-10-01,0.001650
2,2014-11-01,0.001652
3,2014-12-01,0.001669
4,2015-01-01,0.001624


In [36]:
#df_ipc.plot(kind="bar")